# Setup

In [ ]:
import pandas as pd
import scholar_scripts.utils as u
import scholar_scripts.overdrive as od

%load_ext autoreload
%autoreload 2

u.configure_logging()

# Overdrive

## Single Item Search

In [ ]:
title="upstream"
author="mary+oliver"
lib=od.LibraryCode.BOISE
od.search_for_media(author, title, lib)

## Bulk Search

### Build search targets from GoodReads 'to-read' items

In [ ]:
# Go to Good Reads -> My Books -> Import/Export (Under Tools on left ribbon)
# Export the library, this will generate a link for your download

goodreads_filepath = "data/goodreads_export.csv"
gr_library = pd.read_csv(goodreads_filepath)

# TO READ items from your GoodReads list
search_query_str = "`Exclusive Shelf` == 'to-read'"
(search_targets := gr_library.query(search_query_str))

### Build search targets from previous search results

In [ ]:
# TODO replace filepath with your own
df = pd.read_csv("data/generated/overdrive_search_2024-06-16.csv", index_col=0)

(search_targets := df.query(od.Query.AUDIOS_AVAIL))

### Run the Search!

In [ ]:
results = od.search_libraries(search_targets.to_dict(orient="records"), delay=1.5)
search_results = pd.DataFrame(results).set_index("query_title")
df = search_results.sort_index()
df.to_csv(f"data/generated/overdrive_search_{u.now_iso()}.csv", index=True)

## Analyze Results

### Load previous results from save file

In [ ]:
# TODO replace filepath with your own
(df := pd.read_csv("data/generated/overdrive_search_2024-06-16.csv", index_col=0))

#### Find Available Media

In [ ]:
x = [
    {
        "title": title[:25],  # truncate the title ... bloat is real
        "ebook": _df.query("`ebook_available` > 0")["library"].unique(),
        "audio": _df.query("`audiobook_available` > 0")["library"].unique(),
    }
    for title, _df in df[od.SUMMARY_FIELDS].groupby("title")
]

print(pd.DataFrame(x).set_index("title").to_string())